## 2-3. 어휘 집합 구축하기
### BPE 기반 Tokenizer 만들기

### 1단계: 실습 환경 만들기

pip 명령어로 의존성 있는 패키지를 설치합니다.



In [ ]:
!pip install ratsnlp

### 2단계: 구글 드라이브 연동하기


In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

### 3단계: 말뭉치 다운로드 및 전처리

- 코포라(Korpora)를 활용해 BPE 수행 대상 말뭉치를 내려받고 전처리.
- 실습용 말뭉치는 박은정 님이 공개하신 Naver Sentiment Movie Corpus(NSMC)
- 데이터를 내려받아 `nsmc`라는 변수로 읽어들입니다.

In [ ]:
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)

- NSMC에 포함된 영화 리뷰(순수 텍스트)들을 지정된 경로에 저장.

In [ ]:
import os
def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')

write_lines("/content/train.txt", nsmc.train.get_all_texts())
write_lines("/content/test.txt", nsmc.test.get_all_texts())

`train.txt`의 앞부분 확인 - train_data

In [ ]:
!head train.txt

`test.txt`의 앞부분 확인 - test_data

In [ ]:
!head test.txt

### 4단계: GPT 토크나이저 구축

- GPT 계열 모델이 사용하는 토크나이저 ->  Byte-level Byte Pair Encoding(BBPE)입니다.
- 어휘집합 구축 결과를 저장해둘 디렉토리를 구글 드라이브 계정 내 `내 드라이브/nlpbook/bbpe`로 만들어 둠.

In [ ]:
import os
os.makedirs("/gdrive/My Drive/nlpbook/bbpe", exist_ok=True)

- `nsmc` 데이터를 가지고 BBPE 어휘집합을 구축 (시간 조금 소요)

In [ ]:
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size=10000,
    special_tokens=["[PAD]"]
)
bytebpe_tokenizer.save_model("/gdrive/My Drive/nlpbook/bbpe")

- 코드 실행이 끝나면 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/bbpe`)에 `vocab.json`과 `merges.txt`가 생성.
- 전자는 바이트 레벨 BPE의 어휘 집합이며 후자는 바이그램 쌍의 병합 우선순위
- `vocab.json`은 다음과 같이 생김.

In [ ]:
!cat /gdrive/My\ Drive/nlpbook/bbpe/vocab.json

- `merges.txt`의 앞부분은 다음과 같이 생김.

In [ ]:
!head /gdrive/My\ Drive/nlpbook/bbpe/merges.txt

### 5단계: BERT 토크나이저 구축

- BERT는 워드피스(wordpiece) 토크나이저를 사용.
- 우선 어휘집합 구축 결과를 저장해둘 디렉토리를 자신의 구글 드라이브 계정 내 `내 드라이브/nlpbook/bbpe`로 만들어둠.

In [ ]:
import os
os.makedirs("/gdrive/My Drive/nlpbook/wordpiece", exist_ok=True)

- 다음을 수행하면 BERT 모델이 사용하는 워드피스 어휘집합을 구축.(시간이 걸림)

In [ ]:
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size=10000,
)
wordpiece_tokenizer.save_model("/gdrive/My Drive/nlpbook/wordpiece")

- 코드 실행이 끝나면 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 `vocab.txt`가 생성
- `vocab.txt`의 앞부분은 다음과 같이 생김.

In [ ]:
!head /gdrive/My\ Drive/nlpbook/wordpiece/vocab.txt